<a href="https://colab.research.google.com/github/kittyschulz/visual_census/blob/master/Object_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2018 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [0]:
# Copyright 2018 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# Object detection


This Colab demonstrates use of a TF-Hub module trained to perform object detection.

### Imports and function definitions


In [0]:
#@title Imports and function definitions

# Currently %tensorflow_version 2.x installs beta1, which doesn't work here.
# %tensorflow_version can likely be used after 2.0rc0  
!pip install tf-nightly-gpu-2.0-preview

# For running inference on the TF-Hub module.
import tensorflow as tf

import tensorflow_hub as hub

# For downloading the image.
import matplotlib.pyplot as plt
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

# For drawing onto the image.
import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

# For measuring the inference time.
import time

# Check available GPU devices.
print("The following GPU devices are available: %s" % tf.test.gpu_device_name())

### Helper functions for downloading images and for visualization.

Visualization code adapted from [TF object detection API](https://github.com/tensorflow/models/blob/master/research/object_detection/utils/visualization_utils.py) for the simplest required functionality.

In [0]:
def display_image(image):
  fig = plt.figure(figsize=(20, 15))
  plt.grid(False)
  plt.imshow(image)


def download_and_resize_image(url, new_width=256, new_height=256,
                              display=False):
  _, filename = tempfile.mkstemp(suffix=".jpg")
  response = urlopen(url)
  image_data = response.read()
  image_data = BytesIO(image_data)
  pil_image = Image.open(image_data)
  pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS)
  pil_image_rgb = pil_image.convert("RGB")
  pil_image_rgb.save(filename, format="JPEG", quality=90)
  print("Image downloaded to %s." % filename)
  if display:
    display_image(pil_image)
  return filename


def draw_bounding_box_on_image(image,
                               ymin,
                               xmin,
                               ymax,
                               xmax,
                               color,
                               font,
                               thickness=4,
                               display_str_list=()):
  """Adds a bounding box to an image."""
  draw = ImageDraw.Draw(image)
  im_width, im_height = image.size
  (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                ymin * im_height, ymax * im_height)
  draw.line([(left, top), (left, bottom), (right, bottom), (right, top),
             (left, top)],
            width=thickness,
            fill=color)

  # If the total height of the display strings added to the top of the bounding
  # box exceeds the top of the image, stack the strings below the bounding box
  # instead of above.
  display_str_heights = [font.getsize(ds)[1] for ds in display_str_list]
  # Each display_str has a top and bottom margin of 0.05x.
  total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)

  if top > total_display_str_height:
    text_bottom = top
  else:
    text_bottom = bottom + total_display_str_height
  # Reverse list and print from bottom to top.
  for display_str in display_str_list[::-1]:
    text_width, text_height = font.getsize(display_str)
    margin = np.ceil(0.05 * text_height)
    draw.rectangle([(left, text_bottom - text_height - 2 * margin),
                    (left + text_width, text_bottom)],
                   fill=color)
    draw.text((left + margin, text_bottom - text_height - margin),
              display_str,
              fill="black",
              font=font)
    text_bottom -= text_height - 2 * margin


def draw_boxes(image, boxes, class_names, scores, max_boxes=10, min_score=0.1):
  """Overlay labeled boxes on an image with formatted scores and label names."""
  colors = list(ImageColor.colormap.values())

  try:
    font = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf",
                              25)
  except IOError:
    print("Font not found, using default font.")
    font = ImageFont.load_default()

  for i in range(min(boxes.shape[0], max_boxes)):
    if scores[i] >= min_score:
      ymin, xmin, ymax, xmax = tuple(boxes[i])
      display_str = "{}: {}%".format(class_names[i].decode("ascii"),
                                     int(100 * scores[i]))
      color = colors[hash(class_names[i]) % len(colors)]
      image_pil = Image.fromarray(np.uint8(image)).convert("RGB")
      draw_bounding_box_on_image(
          image_pil,
          ymin,
          xmin,
          ymax,
          xmax,
          color,
          font,
          display_str_list=[display_str])
      np.copyto(image, np.array(image_pil))
  return image

### Functions

Pick an object detection module and apply on the downloaded image. Modules:
* **FasterRCNN+InceptionResNet V2**: high accuracy,
* **ssd+mobilenet V2**: small and fast.

In [0]:
module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1" #@param ["https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1", "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"]

detector = hub.load(module_handle).signatures['default']

In [0]:
def load_img(path):
  """loads and decodes image using tf"""
  img = tf.io.read_file(path)
  img = tf.image.decode_jpeg(img, channels=3)
  return img

In [0]:
# Run the detector, single out the car-type objects with some baseline confidence
# crop these objects out and save them with the convention '[i]_[image_name].jpg'

def run_detector(detector, path, confidence=0.25):
  """ 
  run_detector calls load_img() to decode a pillow image, runs our object 
  detector on the image and builds a dictionary with the results. With the 
  input confidence level, the car-type objects detected that fall above 
  that confidence are cropped from the image and saved to the local direct-
  ory calling crop_boxes().
  
  Inputs: 
  detector: the chosen detector
  path: path to image
  confidence: a number between 0 and 1
  
  Outputs:
  None
  Saves images to local directory by calling crop_boxes
  Displays image with annotations if draw_boxes() func uncommented
  """
  img = load_img(path)

  converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
  start_time = time.time()
  result = detector(converted_img)
  end_time = time.time()

  result = {key:value.numpy() for key,value in result.items()}

  # Uncomment below to print number of objects identified and 
  # runtime for each image.
  #print("Found %d objects." % len(result["detection_scores"]))
  #print("Inference time: ", end_time-start_time)
  
  confident_car_boxes = (result["detection_scores"] > confidence) & (result["detection_class_entities"] == str.encode('Car'))
  car_entities = result["detection_class_entities"][confident_car_boxes]
  scores = result["detection_scores"][confident_car_boxes]
  box_coordinates = result["detection_boxes"][confident_car_boxes]
  
  crop_boxes(img.numpy(), box_coordinates, scores, path)
  
  # Uncomment below to see the bounding boxes displayed on the scene
  # MAKE SURE TO COMMENT THESE OUT WHEN RUNNING A BATCH OF IMAGES OR IT WILL TAKE TOO LONG!
  #image_with_boxes = draw_boxes(img.numpy(), box_coordinates, car_entities, scores)
  #display_image(image_with_boxes)

In [0]:
# Crop the car-type objects from the scene based on the bounding box established by the object detector

def crop_boxes(image, boxes, scores, path, max_boxes=10, min_score=0.1):
  """
  crops car-type objects detected which have scores over a given confidence threshold.
  saves the cropped image(s) locally with the naming convention [i]_[img_name].jpg
  
  Inputs:
  image: raw image as a numpy array
  boxes: bounding box coordinates of car-type objects
  scores: numpy arr of values between 0 and 1
  max_boxes: int, default 10
  min_score: float between 0 and 1, default 0.1
  
  Outputs:
  None
  Saves images to local directory
  """
  image_name = path.split('.')[0]
  im_height, im_width, _ = image.shape
  image_pil = Image.fromarray(np.uint8(image)).convert("RGB")
  
  for i in range(min(boxes.shape[0], max_boxes)):
    if scores[i] >= min_score:
      ymin, xmin, ymax, xmax = tuple(boxes[i])
      ymin *= im_height
      ymax *= im_height
      xmin *= im_width
      xmax *= im_width
      
      #if (ymax-ymin)*(xmax-xmin) > (im_height*im_width)*0.025:
        #print('Saving vehicle number {}'.format(i))
      crop_img = image_pil.crop((xmin, ymin, xmax, ymax))
      crop_img.save(str(i) + '_' + image_name + ".jpg", "JPEG")

### Apply Functions to Images

In [0]:
# Upload some scene images we want to get the cars from:

from google.colab import files
uploaded = files.upload()

# Alternatively we can download from Google Drive by uncommenting below:

# import gdown

# url = 'https://drive.google.com/uc?id=1udfCO_bydFKQ1DqSDeXCZzCyXy4AZYYs'
# output = 'image.jpg'
# gdown.download(url, output, quiet=False)

In [0]:
from os import listdir
from os.path import isfile, join

# Get a list of all the uploaded images
images = [f for f in listdir('/content') if f[-3:] == 'jpg']

In [0]:
# Preprocess the image we want to feed into the detector

def resize_image(img, new_width=256, new_height=256,
                              display=False):
  pil_image = Image.open(img)
  pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS)
  pil_image_rgb = pil_image.convert("RGB")
  pil_image_rgb.save(img, format="JPEG", quality=90)
  if display:
    display_image(pil_image)
  return img

In [0]:
# Run the detector on each image we uploaded

for image in images:
  image_path = resize_image(image, 1280, 1024)
  run_detector(detector, image_path)

In [0]:
# Get a list of all the cropped images
images_cropped = [f for f in listdir('/content') if f[1] == '_']

In [0]:
# Display some of the cropped images

# for crop in images_cropped[:3]:
#   image_crop = load_img(crop)
#   display_image(image_crop)

### Save Cropped Images

In [0]:
# zip the cropped images for easy download
import zipfile
zip_file = zipfile.ZipFile('temp.zip', 'w')
for crop in images_cropped:
  zip_file.write(crop, compress_type=zipfile.ZIP_DEFLATED)
zip_file.close()

In [0]:
# Download the cropped images
files.download('temp.zip')